# The Self-Conscious Statisticians

Imagine a group of statisticians who are in a weight loss program.  Because they're statisticians, they want to compute summary statistics over the group weights.  Because they're self-conscious, none want to reveal their weight to the group.  This is a perfect use-case for Cicada and MPC:

In [ ]:
import logging

import numpy

import cicada.additive
import cicada.communicator
import cicada.logging

logging.basicConfig(level=logging.INFO)

@cicada.communicator.NNGCommunicator.run(world_size=5)
def main(communicator):
    log = cicada.logging.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)

    # Each player loads their weight from a file.
    weight = numpy.loadtxt(f"statistician-weight-{communicator.rank}.txt")
        
    # Compute the sum of the player weights.
    mean_share = protocol.share(src=0, secret=protocol.encoder.zeros(shape=()), shape=())
    for rank in communicator.ranks:
        weight_share = protocol.share(src=rank, secret=protocol.encoder.encode(weight), shape=weight.shape)
        mean_share = protocol.add(mean_share, weight_share)
        
    # Divide by the number of players to obtain the mean weight.
    mean_share = protocol.untruncated_private_public_divide(mean_share, communicator.world_size)
    mean_share = protocol.truncate(mean_share)
                                 
    # Reveal the mean weight to the group.
    mean = protocol.encoder.decode(protocol.reveal(mean_share))
                                 
    log.info(f"Mean weight revealed to player {communicator.rank}: {mean}")
    
main();

If we manually inspect the players' weights, we can see that the result is correct, making allowances for the default 16-bit fixed point precision:

In [ ]:
weights = [numpy.loadtxt(f"statistician-weight-{rank}.txt") for rank in range(5)]
print(f"weights: {weights}")
print(f"mean: {numpy.mean(weights)}")